参考 ：https://zhuanlan.zhihu.com/p/42938173

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline 

def draw_hist(img):
    plt.hist(img.ravel(),256,[0,256]);
    plt.show()

# 综述

|算法 | 优点| 缺点|
|:-----|:-------|:----|
|光流场法| 可适用摄像机静止和运动两种场合| 计算量大,易受光照变化影响
帧间差分法| 原理简单,计算量小;  适用于实时系统| 检测目标不完整
背景减法|算法复杂度小,提取目标完整| 背景模型需要实时更新

| |背景差分法| 帧间差分法 |光流法
|:---|:----|:-----|:-----|
运算结果 |运动目标的整个区域|运动目标的外轮廓| 运动目标的整个区域
运算复杂度|由背景建模算法的复杂程度决定| 小| 大
适用范围 | 摄像头固定(背景相对固定)|摄像头固定(背景相对固定)| 摄像头固定摄像头固定/运动,与背景信息无关
鲁棒性| 较好|好|差

总结： 光流场法计算复杂， 不适用于实时监控系统；帧差法计算简单，但是检测结果不完整；背景减法效果较好，然而建立一个良好的背景模型需要花费很大计算量和存储量的开销。

在对视频或图像序列进行分析时，运动目标检测的主要任务是：

- 判断视频图像序列中每一帧有无对应的前景目标
- 能否将对应的前景目标的特征提取出来，并将前景目标显示出来。
只有检测到了前景目标才能对其进行跟踪与后续的相关分析处理，因此目标检测是目标跟踪处理的基础。局部遮挡、光照等因素都会影响目标检测的精度。

# 背景减除

是由当前输入图像片中的像素与背景图像片像素相减得到两者差异的区域，背景指的是去除运动目标那张图像片，所得区域即为需要检测的运动目标区域。

![pic](../assets/dynamic_detect2.jpg)

在很多基础应用中背景减除都是一个非常重要的步骤。例如顾客统计，使用一个静态摄像头来记录进入和离开房间的人数，或者是交通摄像头，需要提取交通工具的信息等。在所有的这些例子中，首先要将人或车单独提取出来。技术上来说，我们需要从静止的背景中提取移动的前景。  

  如果你有一张背景（仅有背景不含前景）图像，比如没有顾客的房间，没有交通工具的道路等，那就好办了。我们只需要在新的图像中减去背景就可以得到前景对象了。但是在大多数情况下，我们没有这样的（背景）图像，所以我们需要从我们有的图像中提取背景。如果图像中的交通工具还有影子的话，那这个工作就更难了，因为影子也在移动，仅仅使用减法会把影子也当成前景。真是一件很复杂的事情。
  
背景减法利用图像序列中的当前帧和事先确定的背景参考模型间的差异比较，来确定运动物体位置，是一种基于统计学原理的运动目标检测的方法。这种方法的性能取决于背景建模技术，Gloyer等人使用单高斯模型的思路，但常常不能准确地描述背景模型。

1999年Stauffer等人提出了经典的混合高斯背景建模法，这种方法不仅对复杂场景的适应强，而且能通过自动计算的模型参数来对背景模型调整，虽然增加了高斯分布的个数，造成计算量增大，但检测速度很快，且检测准确，容易实现。基于混合高斯模型建模的背景减法已是运动检测的**主流方法**。OpenCv中有三种三种比较容易使用的方法。
## 1. BackgroundSubtractorMOG
    
这是一个以混合高斯模型为基础的前景/背景分割算法。它是KadewTraKuPong和R.Bowden在2001年提出的。它使用K（K=3或5）个高斯分布混合对
背景像素进行建模。使用这些颜色（在整个视频中）存在时间的长短作为混合的权重。背景的颜色一般持续的时间最长，而且更加静止。一个像素怎么会有分布呢？在x，y平面上一个像素就是一个像素没有分布，但是我们现在讲的背景建模是基于时间序列的，因此每一个像素点所在的位置在整个时间序列中就会有很多值，从而构成一个分布。

  在编写代码时，我们需要使用函数：`cV2.createBackgroundSubtractorMOGO`创建一个背景对象。这个函数有些可选参数，比如要进行建模场景的时间长度，高斯混合成分的数量，阈值等。将他们全部设置为默认值。然后在整个视频中我们是需要使用`backgroundsubtractor.apply()`就可以得到前景的掩模
了。

### 基础示例

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)  #笔记本摄像头

fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
# 可选参数 比如 进行建模场景的时间长度 高斯混合成分的数量-阈值等

while True:
    ret, frame = cap.read()
    # 计算前景掩码
    fgmask = fgbg.apply(frame)

    cv2.imshow('frame', fgmask)
    k = cv2.waitKey(1)  #& 0xff
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### 检测运动目标

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()

# 可选参数 比如 进行建模场景的时间长度 高斯混合成分的数量-阈值等

kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))  # 定义结构元素
kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8, 3))
color_m = (255, 0, 0)


while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    fgmask = fgbg.apply(frame)
    # 二值化
    thresh = cv2.threshold(fgmask, 80, 255, cv2.THRESH_BINARY)[1]
    th_eroad = cv2.erode(thresh, kernel1, iterations=1)
    th_dilate = cv2.dilate(th_eroad, kernel2, iterations=2)
    # 查找检测物体的轮廓,只检测外轮廓,只需4个点来保存轮廓信息
    contours_m, hierarchy_m = cv2.findContours(th_dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours_m:
        if cv2.contourArea(c) < 300:
            continue
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color_m, 2)

    cv2.imshow("source", frame)
    cv2.imshow("apply", fgmask)
    cv2.imshow('th_eroad', th_eroad)
    cv2.imshow('th_dilate', th_dilate)
    k = cv2.waitKey(20)  #& 0xff
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## 2. BackgroundSubtractorMOG2

这个也是以高斯混合模型为基础的背景/前景分割算法。它是以2004年和2006年Z.Zivkovic的两篇文章为基础的。这个算法的一个**特点是它为每一个像素选择一个合适数目的高斯分布。（上一个方法中我们使用是K高斯分布）**。**这样就会对由于亮度等发生变化引起的场景变化产生更好的适应。**

和前面一样我们需要创建一个背景对象。但在这里我们我们可以选择是否检测阴影。如果 `detectShadous=True`（默认值），它就会检测并将影子标记出来，但是这样做会**降低处理速度**。影子会被标记为灰色。

### 基础示例

In [ ]:
a = cv2.cvtColor(cv2.absdiff(frame, frame), cv2.COLOR_BGR2GRAY)

In [ ]:
cv2.imshow('frame', frame)
k = cv2.waitKey(3000)  #& 0xff
cap.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture('D:\OneDrive - business\git_clone\OpenCV-Python-Tutorial-master\data\street_car.mp4')
# cap = cv2.VideoCapture(0)  #笔记本摄像头

fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)
while True:
    ret, frame = cap.read()
    fgmask = fgbg.apply(frame)

    cv2.imshow('frame', fgmask)
    k = cv2.waitKey(30)  #& 0xff
    if k == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

## 3. BackgroundSubtractorGMG

此算法结合了静态背景图像估计和每个像素的贝叶斯分割。这是2012年Andrew_B.Godbehere,Akihiro Matsukawa和Ken_Goldberg在文章中提出的。

它使用前面很少的图像（默认为前120帧）进行**背景建模**。使用了概率前景估计算法（使用贝叶斯估计鉴定前景）。这是一种自适应的估计，新观察到的对象比旧的对象具有更高的权重，**从而对光照变化产生适应**。一些形态学操作如开运算闭运算等被用来除去不需要的噪音。在前几帧图像中你会得到一个黑色窗口。

对结果进行形态学**开运算对与去除噪声很有帮助**。

### 基础示例

In [ ]:
import numpy as np
import cv2

# cap = cv2.VideoCapture('../data/vtest.avi')
cap = cv2.VideoCapture(
    'D:\OneDrive - business\git_clone\OpenCV-Python-Tutorial-master\data\street_car.mp4'
)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
fgbg = cv2.bgsegm.createBackgroundSubtractorGMG()  #前 120 帧

counter = 0
while True:
    ret, frame = cap.read()
    fgmask = fgbg.apply(frame)
    # 开操作
    fgmask_ = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    cv2.imshow('fgmask', fgmask)
    cv2.imshow('MORPH_OPEN', fgmask_)
    cv2.imshow('src', frame)

    # 前120 帧都是黑的
    counter += 1
    print(counter, end=',')

    k = cv2.waitKey(10)  # & 0xff
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## 三种背景减除综合对比

In [ ]:
import numpy as np
import cv2
import imageio

cap = cv2.VideoCapture("D:\OneDrive - business\git_clone\OpenCV-Python-Tutorial-master\data\street_car.mp4")

fgbg1 = cv2.bgsegm.createBackgroundSubtractorMOG()
fgbg2 = cv2.createBackgroundSubtractorMOG2()
fgbg3 = cv2.bgsegm.createBackgroundSubtractorGMG(60)  # initializationFrames=120

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

# 保存gif参数设置
gif1 = 'D:/Video/v1.gif'
gif2 = 'D:/Video/v2.gif'
gif3 = 'D:/Video/v3.gif'
frames1 = []
frames2 = []
frames3 = []

while True:
    ret, frame = cap.read()
    if not ret:
        print('not found')
        break
    frame = cv2.resize(frame, (400, 400), interpolation=cv2.INTER_CUBIC)

    # 前景掩码
    fgmask1 = fgbg1.apply(frame)
    fgmask2 = fgbg2.apply(frame)
    fgmask3 = fgbg3.apply(frame)

    fgmask4 = cv2.morphologyEx(fgmask3, cv2.MORPH_OPEN, kernel, iterations=1)  # 形态学开运算

    cv2.imshow('MOG', fgmask1)
    cv2.imshow('MOG2', fgmask2)
    cv2.imshow('GMG', fgmask3)
    cv2.imshow('MORPH_OPEN', fgmask4)

    # 加入帧
    frames1.append(fgmask1)
    frames2.append(fgmask2)
    frames3.append(fgmask4)

    k = cv2.waitKey(50) & 0xff
    if k == 27 or k == ord('q'):
        break

# 保存gif
imageio.mimsave(gif1, frames1, 'GIF', duration=1/20)
imageio.mimsave(gif2, frames2, 'GIF', duration=1/20)
imageio.mimsave(gif3, frames3, 'GIF', duration=1/20)

cap.release()
cv2.destroyAllWindows()

---

# 帧间差分法

帧差法依据的原则是：当视频中存在移动物体的时候，相邻帧（或相邻三帧）之间在灰度上会有差别，求取两帧图像灰度差的绝对值，则静止的物体在差值图像上表现出来全是0，而移动物体特别是移动物体的轮廓处由于存在灰度变化为非0，当绝对值超过一定阈值时，即可判断为运动目标，从而实现目标的检测功能。 帧间差分法的优点是算法实现简单，程序设计复杂度低；对光线等场景变化不太敏感，能够适应各种动态环境，有着比较强的鲁棒性。缺点是不能提取出对象的完整区域，对象内部有“空洞”，只能提取出边界，边界轮廓比较粗，往往比实际物体要大。对快速运动的物体，容易出现鬼影的现象，甚至会被检测为两个不同的运动物体，对慢速运动的物体，当物体在前后两帧中几乎完全重叠时，则检测不到物体。故该方法一般适用于简单的实时运动检测的情况。 这里介绍二帧法和三帧法的实现。

![pic](../assets/dynamic_detect1.jpg)

## 二差分法

相邻帧间差分法直接对相邻的两帧图像做差分运算，并取差分运算的绝对值构成移动物体，

优点：
- 运算快速
- 实时性高

缺点：
- 无法应对光照的突变
- 物体间一般具有空洞

步骤：

>前一帧 | 当前帧 ➡ cv2.absdiff() ➡ 灰度图 ➡ 二值化 ➡ (形态学变换) ➡ 轮廓检测 ➡ 过滤轮廓面积

In [7]:
# 导入必要的软件包
import cv2

# 视频文件输入初始化
filename = "D:\OneDrive - business\git_clone\OpenCV-Python-Tutorial-master\data\street_car.mp4"
camera = cv2.VideoCapture(filename)

# # 视频文件输出参数设置
out_fps = 12.0  # 输出文件的帧率
fourcc = cv2.VideoWriter_fourcc('M', 'P', '4', '2')
out1 = cv2.VideoWriter('D:/video/v1.avi', fourcc, out_fps, (500, 400))

# 计算时钟
e1 = cv2.getTickCount()

# 第一帧进行初始化
_, lastFrame = camera.read()
lastFrame = cv2.GaussianBlur(lastFrame, (7, 7), 0)


# 遍历视频的每一帧
while camera.isOpened():
    # 读取下一帧
    (ret, frame) = camera.read()

    # 如果不能抓取到一帧，说明我们到了视频的结尾
    if not ret:
        break
        
#     frame = cv2.GaussianBlur(frame, (3, 3), 0)

    # 调整该帧的大小
#     frame = cv2.resize(frame, (500, 400), interpolation=cv2.INTER_CUBIC)

# 计算当前帧和前帧的不同
    frameDelta = cv2.absdiff(lastFrame, frame)

    # 当前帧设置为下一帧的前帧
    lastFrame = frame.copy()

    # 结果转为灰度图
    thresh = cv2.cvtColor(frameDelta, cv2.COLOR_BGR2GRAY)

    # 图像二值化
    thresh = cv2.threshold(thresh, 25, 255, cv2.THRESH_BINARY)[1]
    
    #去除图像噪声,先腐蚀再膨胀(形态学开运算)
#     thresh = cv2.erode(thresh, None, iterations=1)
#     thresh = cv2.dilate(thresh, None, iterations=2)

    # 阀值图像上的轮廓位置
    cnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                                       cv2.CHAIN_APPROX_SIMPLE)

    # 遍历轮廓
    for c in cnts:
        # 忽略小轮廓，排除误差
        if cv2.contourArea(c) < 300:
            continue

        # 计算轮廓的边界框，在当前帧中画出该框
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 显示当前帧
    cv2.imshow("frame", frame)
    cv2.imshow("frameDelta", frameDelta)
    cv2.imshow("thresh", thresh)

    # 保存视频
    out1.write(frame)
    
    # 如果q键被按下，跳出循环
    if cv2.waitKey(50) & 0xFF == ord('q'):
        break
    
e2 = cv2.getTickCount()
print((e2 - e1) / cv2.getTickFrequency())

# 清理资源并关闭打开的窗口
out1.release()
camera.release()
cv2.destroyAllWindows()

2.3082306


## 三帧差分法

　　三帧差法是在相邻帧差法基础上改进的算法，在一定程度上优化了运动物体双边，粗轮廓的现象，相比之下，三帧差法比相邻帧差法更适用于物体移动速度较快的情况，比如道路上车辆的智能监控。
　　三帧差法基本实现步骤如下：首先前两帧图像做灰度差，然后当前帧图像与前一帧图像做灰度差，最后1和2的结果图像按位做“与”操作，进行阙值判断和得出运动目标。
  
>相邻三帧 ➡ △1=absdiff(帧1, 帧2)、△2=absdiff(帧2, 帧3) ➡ bitwise_and(△1, △2) ➡ 灰度图 ➡ 二值化 ➡ (形态学变换) ➡ 轮廓检测 ➡ 过滤轮廓面积


In [184]:
# 导入必要的软件包
import cv2

# 视频文件输入初始化
filename = "D:\OneDrive - business\git_clone\OpenCV-Python-Tutorial-master\data\street_car.mp4"
camera = cv2.VideoCapture(filename)

KERNEL1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
KERNEL2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8, 3))
e1 = cv2.getTickCount()
# 初始化当前帧的前两帧,计算第一二帧的不同
_, lastFrame1 = camera.read()
_, lastFrame2 = camera.read()
frameDelta1 = cv2.absdiff(lastFrame1, lastFrame1)  # 帧差一


while camera.isOpened():
    # 读取下一帧
    (ret, frame) = camera.read()

    # 如果不能抓取到一帧，说明我们到了视频的结尾
    if not ret:
        break

#     # 调整该帧的大小
#     frame = cv2.resize(frame, (500, 400), interpolation=cv2.INTER_CUBIC)

    # 计算当前帧和前帧的不同,计算三帧差分
    frameDelta2 = cv2.absdiff(lastFrame2, frame)  # 帧差二
    thresh = cv2.bitwise_or(frameDelta1, frameDelta2)  # 图像与运算
    thresh2 = thresh.copy()

    # 当前帧设为下一帧的前帧,前帧设为下一帧的前前帧,帧差二设为帧差一
    lastFrame1 = lastFrame2
    lastFrame2 = frame.copy()
    frameDelta1 = frameDelta2

    # 结果转为灰度图
    thresh = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)

    # 图像二值化
    thresh = cv2.threshold(thresh, 25, 255, cv2.THRESH_BINARY)[1]

    # 开运算
    thresh = cv2.erode(thresh, KERNEL1, iterations=1)
    thresh = cv2.dilate(thresh, KERNEL2, iterations=3)



    # 阀值图像上的轮廓位置
    cnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 遍历轮廓
    for c in cnts:
        # 忽略小轮廓，排除误差
        if cv2.contourArea(c) < 300:
            continue

        # 计算轮廓的边界框，在当前帧中画出该框
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 显示当前帧
    cv2.imshow("frame", frame)
    cv2.imshow("diff", thresh)
    cv2.imshow("threst2", thresh2)
    
    # 如果q键被按下，跳出循环
    if cv2.waitKey(2000) & 0xFF == ord('q'):
        break

e2 = cv2.getTickCount()
print((e2 - e1) / cv2.getTickFrequency())
    
# 清理资源并关闭打开的窗口
camera.release()
cv2.destroyAllWindows()

79.5002203


# KNN

　kNN 可以说是最简单的监督学习分类器了。想法也很简单，就是找出测试数据在特征空间中的最近邻居。

In [ ]:
import cv2 as cv
import numpy as np


# 设置文件
file_test = "D:\OneDrive - business\git_clone\OpenCV-Python-Tutorial-master\data\street_car.mp4"
cap = cv.VideoCapture(0)

# 设置变量
video_skip = True
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))  # 定义卷积核
color_m = (255, 0, 0)

# 背景差法
knn = cv.createBackgroundSubtractorKNN(history=60, dist2Threshold=False, detectShadows=False)

while True:
    # 读取一帧
    ret, frame = cap.read()
    # 如果视频结束，跳出循环
    if not ret:
        break

    # 计算前景掩码
    fgmask = knn.apply(frame)
    draw1 = cv.threshold(fgmask, 25, 255, cv.THRESH_BINARY)[1]  # 二值化
#     print((fgmask==draw1).all())
    # 开操作
    draw1 = cv.morphologyEx(draw1, cv.MORPH_OPEN, kernel, iterations=1)

    # 查找检测物体的轮廓,只检测外轮廓,只需4个点来保存轮廓信息
    contours_m, hierarchy_m = cv.findContours(draw1, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    for c in contours_m:
        if cv.contourArea(c) < 300:
            continue
        (x, y, w, h) = cv.boundingRect(c)
        cv.rectangle(frame, (x, y), (x + w, y + h), color_m, 2)

    cv.imshow("source", frame)
    cv.imshow("apply", fgmask)
    cv.imshow("draw", draw1)
    k = cv.waitKey(30)
    if k == ord('q'):
        break


cap.release()
cv.destroyAllWindows()

# 光流法

在实际测试中，虽然它能检测运动目标的整个区域，适用于摄像机静止和运动情况，但大多数的光流计算方法计算量巨大，结构复杂，且易受光照、物体遮挡或图像噪声的影响，鲁棒性差，故一般不被对精度和实时性要求比较高的监控系统所采用

- https://zhuanlan.zhihu.com/p/42942198

- https://blog.csdn.net/ssybc/article/details/89637951



# 其他Demo

## 仅检测是否有入侵

通过2帧差法，判断是否大于阈值，进行抓拍保存，不进行动态区域标记

In [ ]:
"""
http://www.technicdynamic.com/2017/08/28/python-motion-detection-with-opencv-simple/
"""
import cv2
from datetime import datetime

threshold = 110000  # Threshold for triggering "motion detection"


def diffImg(t0, t1, t2):  # Function to calculate difference between images.
    d1 = cv2.absdiff(t2, t1)
    d2 = cv2.absdiff(t1, t0)
    return cv2.bitwise_and(d1, d2)


cam = cv2.VideoCapture(0)  # Lets initialize capture on webcam

# Read three images first:
t_minus = cv2.cvtColor(cam.read()[1], cv2.COLOR_RGB2GRAY)
t = cv2.cvtColor(cam.read()[1], cv2.COLOR_RGB2GRAY)
t_plus = cv2.cvtColor(cam.read()[1], cv2.COLOR_RGB2GRAY)
# Lets use a time check so we only 每秒处理一帧
timeCheck = datetime.now().second

print('frame size: {}'.format(t_minus.shape[0] * t_minus.shape[1]))

while True:
    ret, frame = cam.read()
    cv2.imshow('Indicator', frame)
    # every second
    if timeCheck != datetime.now().second:
        t_minus = t
        t = t_plus
        t_plus = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        diff_img = diffImg(t_minus, t, t_plus)
        count = cv2.countNonZero(diff_img)
        if count > threshold:
            print('{}s Detected {}'.format(timeCheck, count))
            cv2.imshow('diff', diff_img)
#             cv2.imwrite(datetime.now().strftime('%Y%m%d_%Hh%Mm%Ss%f') + '.jpg', dimg)
        timeCheck = datetime.now().second

    key = cv2.waitKey(10)
    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()  # comment to hide window